In [1]:
import pandas as pd
from astropy.time import Time
import spiceypy as spy
import numpy as np
import rebound as rb
import matplotlib.pyplot as plt
from Utils import *
from astroquery.jplhorizons import Horizons

In [2]:
fireballs = pd.read_csv('datos/cneos_fireball_data_original.csv', comment='#')
fireballs

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,2023-08-05 19:54:50,19.9N,131.7W,44.0,21.8,3.3,20.4,-6.8,1.719000e+12,3.800
1,2023-07-26 03:41:54,18.9N,103.4W,36.1,15.6,-9.5,12.2,1.9,1.510000e+11,0.440
2,2023-07-07 16:15:31,14.5N,126.6W,38.3,25.7,2.6,4.1,-25.2,2.600000e+10,0.092
3,2023-07-06 05:52:31,59.3S,145.4W,71.0,NaN,NaN,NaN,NaN,3.000000e+10,0.100
4,2023-06-21 19:39:13,12.4N,62.8E,40.4,16.5,7.3,-2.0,-16.4,5.430000e+11,1.400
...,...,...,...,...,...,...,...,...,...,...
955,1993-11-29 17:48:41,26.5N,78.3E,NaN,NaN,NaN,NaN,NaN,2.600000e+10,0.092
956,1993-10-31 03:39:27,51.3N,100.9W,NaN,NaN,NaN,NaN,NaN,4.000000e+10,0.130
957,1991-10-04 09:22:47,78.7N,6.3E,NaN,NaN,NaN,NaN,NaN,5.500000e+11,1.400
958,1990-10-01 03:51:47,7.5N,142.8E,NaN,NaN,NaN,NaN,NaN,2.500000e+12,5.200


In [3]:
fireballs = fireballs.dropna()

In [4]:
def change_coord(x):
    if x[-1] == 'N' or x[-1] == 'E':
        new = float(x[:-1])
    elif x[-1] == 'S' or x[-1] == 'W':
        new = -float(x[:-1])
    return new    

In [ ]:
fireballs['Longitude (deg.)'] = fireballs['Longitude (deg.)'].apply(change_coord)
fireballs['Latitude (deg.)'] = fireballs['Latitude (deg.)'].apply(change_coord)

C:\Users\aguju\AppData\Local\Temp\ipykernel_5340\2173166978.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fireballs['Longitude (deg.)'] = fireballs['Longitude (deg.)'].apply(change_coord)
C:\Users\aguju\AppData\Local\Temp\ipykernel_5340\2173166978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fireballs['Latitude (deg.)'] = fireballs['Latitude (deg.)'].apply(change_coord)


In [6]:


!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls -O naif0012.tls
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/pck00010.tpc -O pck00010.tpc
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_fixed.tf -O earth_fixed.tf
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_720101_230601.bpc -O earth_720101_230601.bpc
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de438.bsp -O de438.bsp
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_latest_high_prec.bpc -O earth_latest_high_prec.bpc


'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [36]:
spy.furnsh(['naif0012.tls', 'pck00010.tpc', 'earth_fixed.tf', 'earth_720101_230601.bpc', 'de438.bsp', 'earth_latest_high_prec.bpc'])

In [10]:
lat = fireballs['Latitude (deg.)'].to_numpy()
lon = fireballs['Longitude (deg.)'].to_numpy()
alt = fireballs['Altitude (km)'].to_numpy()
date = fireballs['Peak Brightness Date/Time (UT)'].to_numpy()


In [11]:
x_earth_ecl = []
y_earth_ecl = []
z_earth_ecl = []
for i in range(len(lat)):
    
    r_earth_ecl = Geo2Eclip(lat[i],lon[i],alt[i],date[i]) 
    
    x_earth_ecl.append(r_earth_ecl[0])
    y_earth_ecl.append(r_earth_ecl[1])
    z_earth_ecl.append(r_earth_ecl[2])

In [12]:
fireballs['x ecliptic'] = x_earth_ecl
fireballs['y ecliptic'] = y_earth_ecl
fireballs['z ecliptic'] = z_earth_ecl

/tmp/ipykernel_26040/1318801435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fireballs['x ecliptic'] = x_earth_ecl
/tmp/ipykernel_26040/1318801435.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fireballs['y ecliptic'] = y_earth_ecl
/tmp/ipykernel_26040/1318801435.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

### Comparar integración del bólido '2014-01-08 17:05:34' con el paper de Siraj

In [13]:
meteor = fireballs.loc[fireballs['Peak Brightness Date/Time (UT)'].str.contains('2014-01-08 17:05:34')]

In [14]:
print('Object : ')
meteor

Object : 


,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt),x ecliptic,y ecliptic,z ecliptic
355,2014-01-08 17:05:34,-1.3,147.6,18.7,44.8,-3.4,-43.5,-10.3,3.100000e+10,0.11,-5393.678116,1091.817339,3249.252992


In [15]:
#PARAMETERS 

date = meteor['Peak Brightness Date/Time (UT)'][355]
lon = meteor['Longitude (deg.)'][355]
lat = meteor['Latitude (deg.)'][355]
alt = meteor['Altitude (km)'][355]
vx = meteor['vx'][355]
vy = meteor['vy'][355]
vz = meteor['vz'][355]
x_eclip = meteor['x ecliptic'][355]
y_eclip = meteor['y ecliptic'][355]
z_eclip = meteor['z ecliptic'][355]

In [16]:
#meteor's pre-impact velocity in a geocentric Earth-fixed reference frame
v = np.array([vx, vy, vz])   

T_earth = 86400

#vector velocidad de rotación
omega = np.array([0,0,(2*np.pi)/T_earth]) 

print('Velocidad del asteroide antes del impacto respecto a la tierra ')
print(v, mag(v))  


Velocidad del asteroide antes del impacto respecto a la tierra 
[ -3.4 -43.5 -10.3] 44.83190827970632


In [17]:
#posición del meteoro en coordenadas rectangulares respecto a la tierra 
r = Geo2Rec(lon, lat, alt)
print(r)

r_iner = np.array([x_eclip, y_eclip, z_eclip]) 
print(r_iner)


[-5399.64687765  3426.72010848  -144.1587164 ]
[-5393.67811613  1091.81733861  3249.25299229]


In [18]:
v_E = (v - spy.vcrss(omega, r))
v_E, -v, mag(v_E), np.arccos((v@r)/(np.linalg.norm(v)*np.linalg.norm(r)))*180/np.pi


(array([ -3.15080188, -43.1073266 , -10.3       ]),
 array([ 3.4, 43.5, 10.3]),
 44.4326361915465,
 116.78101435530589)

In [19]:
et = spy.utc2et(date)
mx = spy.pxform('IAU_EARTH', 'ECLIPJ2000', et)
print("Matriz de rotación ")
print(mx)

r_eclip = spy.mxv(mx, r)
v_eclip = spy.mxv(mx, v_E)

print(f"Posición asteroide inercial ecliptico: {r_eclip}")
print(f"Velocidad asteroide inercial ecliptico: {v_eclip}")
print(f"Magnitud velocidad: {mag(v_eclip)}")

Matriz de rotación 
[[ 0.99716246 -0.07526728  0.00136306]
 [ 0.06851585  0.91492118  0.39777482]
 [-0.03118652 -0.39655273  0.91748206]]
Posición asteroide inercial ecliptico: [-5642.44156711  2707.87470299 -1322.7420744 ]
Velocidad asteroide inercial ecliptico: [  0.08867015 -43.75276672   7.7425255 ]
Magnitud velocidad: 44.4326361915465


In [20]:
Horizons(id = '399', location='@0', epochs = dict(start = '2014-03-21', stop = '2014-03-22', step = '1d')).vectors()

targetname,datetime_jd,datetime_str,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str11,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64
Earth (399),2456737.5,A.D. 2014-Mar-21 00:00:00.0000,-0.9945468211497613,-0.003722207387582296,-0.0001020957168866397,-0.0002507165931815516,-0.01726151078274061,-1.703512565068483e-07,0.005744063655928266,0.9945537917631541,0.0003153176165069349
Earth (399),2456738.5,A.D. 2014-Mar-22 00:00:00.0000,-0.9946489028189718,-0.0209832991638009,-0.0001022309209574547,4.656936758353987e-05,-0.01725985104532338,-9.456438268899956e-08,0.005745891177136442,0.9948702172514479,0.0003174770455735695


In [21]:
sim = rb.Simulation()
sim.units = 'km', 's', 'kg'
sim.integrator = "ias15"

sun = sim.add("Sun", hash='sun', date=date)
earth = sim.add("Earth", hash='earth', date=date)

r_earth = np.array(sim.particles['earth'].xyz)
v_earth = np.array(sim.particles['earth'].vxyz)

r_asteroid = r_eclip  + r_earth 
v_asteroid = v_eclip + v_earth #así si es 

#v_asteroid = -v_eclip + v_earth #esto es errado 
v_esc = (2*sim.G*1.98e30/((r_asteroid@r_asteroid)**0.5))**0.5

asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2], 
                   vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2],)
r_asteroid, v_asteroid, r_earth, v_earth, v_esc, mag(v_asteroid)

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')


(array([-4.56682168e+07,  1.39459114e+08, -1.96792418e+04]),
 array([-28.69279537, -53.1376722 ,   7.74260786]),
 array([-4.56625744e+07,  1.39456406e+08, -1.83564997e+04]),
 array([-2.87814655e+01, -9.38490548e+00,  8.23593561e-05]),
 42.438492349511534,
 60.883796606945346)

In [42]:
v_esc_E = (2*sim.G*5.98e24/((r@r)**0.5))**0.5
v_mag = mag(v_eclip)
v_inf_E = (v_mag**2 - v_esc_E**2)**0.5
v_esc_E, v_inf_E, v_mag

(11.170665203719915, 43.0055275265486, 44.4326361915465)

In [43]:
sim.status()

---------------------------------
REBOUND version:     	3.23.5
REBOUND built on:    	Mar  2 2023 13:22:06
Number of particles: 	3
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.particle.Particle object at 0x7fcb7ff536c0, m=1.9884754159566474e+30 x=154505.2265538892 y=-338056.7291831016 z=-14068.46131306503 vx=0.01044665676699867 vy=0.004186721288873228 vz=-0.0002423673964135767>
<rebound.particle.Particle object at 0x7fcb7ff521c0, m=6.045825576341311e+24 x=-45662574.40348314 y=139456406.3682046 z=-18356.49972175807 vx=-28.78146551981982 vy=-9.384905480708625 vz=8.235935607459055e-05>
<rebound.particle.Particle object at 0x7fcb7ff536c0, m=0.0 x=-45668216.84505025 y=139459114.24290758 z=-19679.241796156795 vx=-28.692795370927715 vy=-53.1376721976239 vz=7.74260786142672>
---------------------------------


In [44]:
sim.move_to_com()
o = sim.calculate_orbits(primary=sun)
o[1].e, o[1].a/1.496e8  #Verdaderos elementos orbitales (correctos)

(2.3735204241631296, -0.46600352883966073)

In [ ]:
state = [r_eclip[205][0], r_eclip[205][1], r_eclip[205][2], v_eclip[205][0], v_eclip[205][1], v_eclip[205][2]]
et = spy.utc2et(date[205])
sim = rb.Simulation()
sim.units = 'km', 's', 'kg'
mu = 1.98847e30*sim.G
 

In [25]:
r_eclip, v_eclip
state = [r_eclip[0], r_eclip[1], r_eclip[2], v_eclip[0], v_eclip[1], v_eclip[2]]
mu = 1.98847e30*sim.G

In [26]:
spy.oscelt(state, et, mu)

array([2.41464109e-01, 9.99924509e-01, 2.27512896e-01, 3.84440267e+00,
       1.98165079e+00, 3.15413678e+00, 4.42472801e+08, 1.32712079e+11])

In [27]:
r_eclip, v_eclip
state = [r_asteroid[0], r_asteroid[1], r_asteroid[2], v_asteroid[0], v_asteroid[1], v_asteroid[2]]
mu = 1.98847e30*sim.G

In [28]:
spy.oscelt(state, et, mu)

array([ 9.54224324e+07,  2.36477371e+00,  1.74822513e-01,  1.88801691e+00,
        1.04370242e+00, -1.23379674e+00,  4.42472801e+08,  1.32712079e+11])

### corregir constantes

In [46]:
sim = rb.Simulation()
sim.units = 'km', 's', 'kg'
sim.integrator = "ias15"

sun = sim.add("Sun", hash='sun', date=date)
earth = sim.add("Earth", hash='earth', date=date)

r_earth = np.array(sim.particles['earth'].xyz)
v_earth = np.array(sim.particles['earth'].vxyz)

r_asteroid = r_eclip  + r_earth 
#v_asteroid = v_eclip + v_earth #así si es 

v_asteroid = -v_eclip + v_earth #esto es errado 
v_esc = (2*sim.G*1.98e30/((r_asteroid@r_asteroid)**0.5))**0.5

asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2], 
                   vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2],)
r_asteroid, v_asteroid, r_earth, v_earth, v_esc, mag(v_asteroid)

sim.move_to_com()
o = sim.calculate_orbits(primary=sun)
o[1].e, o[1].a/1.496e8 

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')


(2.3735204241631296, -0.46600352883966073)

In [65]:
sim.status()

---------------------------------
REBOUND version:     	3.23.5
REBOUND built on:    	Mar  2 2023 13:22:06
Number of particles: 	2
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.particle.Particle object at 0x7f9b1e6719c0, m=1.9884754159566474e+30 x=0.0 y=-5.820766091346741e-11 z=1.8189894035458565e-12 vx=0.0 vy=0.0 vz=0.0>
<rebound.particle.Particle object at 0x7f9b54b49240, m=0.0 x=-45822722.07160414 y=139797170.9720907 z=-5610.780483091763 vx=-0.01044665676699867 vy=-0.004186721288873228 vz=0.0002423673964135767>
---------------------------------


In [31]:
v_mag = (v_asteroid@v_asteroid)**0.5
v_inf = (v_mag**2 - v_esc**2)**0.5
v_inf

43.65559593427102

In [37]:
o = sim.calculate_orbits(primary='sun')
o[0].e

nan

In [95]:
v_earth, v_asteroid, (v_asteroid@v_asteroid)**0.5, (2*sim.G*1.98e30/((r_earth@r_earth)**0.5))**0.5

(array([-2.87814655e+01, -9.38490548e+00,  8.23593561e-05]),
 array([-28.87013567,  34.36786124,  -7.74244314]),
 45.54755806076936,
 42.439118397423954)

In [18]:
v_iner = spy.mxv(mx, v_E - spy.vcrss(omega_vec, r_E))
r_iner = np.array([x_eclip, y_eclip, z_eclip]) 

In [19]:
sim = rb.Simulation()
sim.integrator = "ias15"

sim.add("Sun",hash='sun')
sim.add("Earth",hash='earth')

r_earth = sim.particles['earth'].xyz
v_earth = sim.particles['earth'].vxyz

r_asteroid = r_iner + r_earth
v_asteroid = v_iner + v_earth

v_esc = (2*sim.G*1.98e30/((r_asteroid@r_asteroid)**0.5))**0.5

sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2])


Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')
